In [30]:
import numpy as np
import random
import platform

class Cube:
    def __init__(self):
        self._u = np.matrix([['🟨', '🟨'], ['🟨', '🟨']], dtype = 'str')      
        self._d = np.matrix([['⬜', '⬜'], ['⬜', '⬜']], dtype = 'str')     
        self._f = np.matrix([['🟩', '🟩'], ['🟩', '🟩']], dtype = 'str')
        self._b = np.matrix([['🟦', '🟦'], ['🟦', '🟦']], dtype = 'str')
        self._r = np.matrix([['🟧', '🟧'], ['🟧', '🟧']], dtype = 'str')
        self._l = np.matrix([['🟥', '🟥'], ['🟥', '🟥']], dtype = 'str')
        self._movestack = []
    
    
    def __repr__(self):
        # adjust space size in output depending on platform
        if platform.system() == 'Linux':
            spaces = '   '
        else:
            spaces = '    '
            
        outstr = ''
        for i in range(2):
            outstr += spaces
            for j in range(2):
                outstr += self._u[i,j]
            outstr += '\n'
        
        for i in range(2):
            for j in range(2):
                outstr += self._l[i, j]
            for j in range(2):
                outstr += self._f[i, j]
            for j in range(2):
                outstr += self._r[i, j]
                
            outstr += '\t'
            for j in range(2):
                outstr += self._b[i, j]
            outstr += '\n'
            
        for i in range(2):
            outstr += spaces
            for j in range(2):
                outstr += self._d[i,j]
            outstr += '\n'
        
        return outstr

    
    # 2 = double move, 3 = counter-clockwise move
    def move(self, mv, modifier = 1):
        for i in range(modifier):
            mv()
        self._movestack.append((mv, modifier))
        
    
    # Computes a move from the passed mvstring.  mvstring modifier must be an int 1-3
    def move_from_string(self, mvstring):
        modifier = int(mvstring[1])
        mv = None
        if mvstring[0].lower() == 'r':
            mv = self.R
        elif mvstring[0].lower() == 'l':
            mv = self.L
        elif mvstring[0].lower() == 'u':
            mv = self.U
        elif mvstring[0].lower() == 'd':
            mv = self.D
        elif mvstring[0].lower() == 'f':
            mv = self.F
        elif mvstring[0].lower() == 'b':
            mv = self.B
        for i in range(modifier):
            mv()
            
        self._movestack.append((mv, modifier))
    
    
    # returns stack containing all previous moves performed
    def get_movestack(self):
        return self._movestack
    
    
    # undoes the previous move and calls pop() on the move stack
    def undo(self):
        self._movestack.pop()
    
    
    # Turns layer 90 degrees clockwise with respect to the face.
    def U(self):
        temp = self._l[0].copy()
        self._l[0] = self._f[0]
        self._f[0] = self._r[0]
        self._r[0] = self._b[0]
        self._b[0] = temp
        self._u = np.rot90(self._u, 3)

        
    def D(self):
        temp = self._f[1].copy()
        self._f[1] = self._l[1]
        self._l[1] = self._b[1]
        self._b[1] = self._r[1]
        self._r[1] = temp
        self._d = np.rot90(self._d, 3)        
        
    
    def F(self):
        temp = self._u[1,:].copy()
        self._u[1,:] = np.flip(np.transpose(self._l[:,1]), axis = 1)
        self._l[:,1] = np.flip(np.transpose(self._d[0,:]), axis = 1)
        self._d[0,:] = np.flip(np.transpose(self._r[:,0]), axis = 1)
        self._r[:,0] = np.flip(np.transpose(temp), axis = 1)
        self._f = np.rot90(self._f, 3)
        
        
    def B(self):
        temp = self._u[0,:].copy()
        self._u[0,:] = np.transpose(self._r[:,1])
        self._r[:,1] = np.flip(np.transpose(self._d[1,:]), axis = 0)
        self._d[1,:] = np.transpose(self._l[:,0])
        self._l[:,0] = np.flip(np.transpose(temp), axis = 0)
        self._b = np.rot90(self._b, 3)
        
        
    def R(self):
        temp = self._u[:,1].copy()
        self._u[:,1] = self._f[:,1]
        self._f[:,1] = self._d[:,1]
        self._d[:,1] = np.flip(self._b[:,0], axis=0)
        self._b[:,0] = temp[::-1]    
        self._r = np.rot90(self._r, 3)
        
        
    def L(self):
        temp = self._d[:,0].copy()
        self._d[:,0] = self._f[:,0]
        self._f[:,0] = self._u[:,0]
        self._u[:,0] = np.flip(self._b[:,1], axis=0)
        self._b[:,1] = temp[::-1]    
        self._l = np.rot90(self._l, 3)
    
    
    #X, Y, Z rotate entire cube
    def X(self):
        self.R()
        self.move(self.L, 3)
    
    
    def Y(self):
        self.U()
        self.move(self.D, 3)
        
        
    def Z(self):
        self.F()
        self.move(self.B, 3)
    
     
    # Generates scramble with no consecutive repeating moves.  
    # R, U, F moves on a 2x2 are enough to fully scramble
    def scramble(self, length):
        print_str = 'Your scramble: (Z2) '
        modifiers =({"1" : "", "2" : "2", "3" : "\'"})
        move = ''  
        prev = ''
        for i in range(length):        
            modifier = random.choice(list(modifiers.keys()))
            while move == prev:
                move = random.choice(['r', 'u', 'f'])
            prev = move
            
            if move == 'r':
                self.move(self.R, int(modifier))
            elif move == 'u':
                self.move(self.U, int(modifier))
            elif move == 'f':
                self.move(self.F, int(modifier))
                
            print_str += move.upper() + modifiers[modifier] + " "
        print(print_str)
        
        # re-initialize move stack
        self._movestack = []
        
    
    # Checks if a specific face is solved
    def face_solved(self, face):
        face_flattened = np.ravel(face)
        if len({i for j in face_flattened for i in j}) == 1:
            return True
        return False
        
    
    # Checks if the R, U, F faces are solved
    def solved(self):
        if (
            self.face_solved(self._u) 
            and self.face_solved(self._r) 
            and self.face_solved(self._f)
        ):
            return True
        return False
    
    
    # Returns current state of cube as a 1D string in the format UUUU,DDDD,FFFF,BBBB,RRRR,LLLL
    def to_1d_string(self):
        outstring = ''
        for i in np.ravel(self._u):
            outstring += i
        outstring += ','
        for i in np.ravel(self._d):
            outstring += i
        outstring += ','
        for i in np.ravel(self._f):
            outstring += i
        outstring += ','
        for i in np.ravel(self._b):
            outstring += i
        outstring += ','
        for i in np.ravel(self._r):
            outstring += i
        outstring += ','
        for i in np.ravel(self._l):
            outstring += i
        
        return outstring        
    

In [28]:
from IPython.display import clear_output

class main:   
    
    cube = Cube()
    print(cube)
    
    choice = str.lower(input("Choose a move >>> "))
    while(not choice == 'quit'):
        if len(choice) == 2:
            if choice[len(choice)-1] == '\'':
                modifier = 3
            elif choice[len(choice)-1] == '2':
                modifier = 2
        else:
            modifier = 1

        if choice[0] == 'u':
            cube.move(cube.U, modifier)
        elif choice[0] == 'd':
            cube.move(cube.D, modifier)
        elif choice[0] == 'f':
            cube.move(cube.F, modifier)
        elif choice[0] == 'b':
            cube.move(cube.B, modifier)
        elif choice[0] == 'r':
            cube.move(cube.R, modifier)
        elif choice[0] == 'l':
            cube.move(cube.L, modifier)
        elif choice[0] == 'x':
            cube.move(cube.X, modifier)
        elif choice[0] == 'y':
            cube.move(cube.Y, modifier)
        elif choice[0] == 'z':
            cube.move(cube.Z, modifier)
        elif choice == 'scramble':
            cube = Cube()
            cube.scramble(10)      
        elif choice == 'prev':
            for i in cube.get_movestack():
                print(i[0].__name__ + str(i[1]))

        # overwrite previous console output
        if not (choice == 'scramble' or choice == 'prev'):
            clear_output()
        
        print(cube)
        if(cube.solved()):
            print('Solved!')
            
        choice = str.lower(input("Chose a move >>> "))

   🟨🟨
   🟨🟨
🟥🟥🟩🟩🟧🟧	🟦🟦
🟥🟥🟩🟩🟧🟧	🟦🟦
   ⬜⬜
   ⬜⬜

Choose a move >>> quit


In [31]:
cube = Cube()
print(cube)
cube.move_from_string('R3')
print(cube)
cube.move_from_string('u1')
print(cube)

for i in cube.get_movestack():
    print(i[0].__name__ + str(i[1]))

   🟨🟨
   🟨🟨
🟥🟥🟩🟩🟧🟧	🟦🟦
🟥🟥🟩🟩🟧🟧	🟦🟦
   ⬜⬜
   ⬜⬜

   🟨🟦
   🟨🟦
🟥🟥🟩🟨🟧🟧	⬜🟦
🟥🟥🟩🟨🟧🟧	⬜🟦
   ⬜🟩
   ⬜🟩

   🟨🟨
   🟦🟦
🟩🟨🟧🟧⬜🟦	🟥🟥
🟥🟥🟩🟨🟧🟧	⬜🟦
   ⬜🟩
   ⬜🟩

R3
U1
